In [ ]:
import pandas as pd
import numpy as np
import time
from copy import deepcopy
from pprint import pprint



from sklearn import cross_validation, metrics
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import train_test_split
from evaluate_metrics import plot_learning_curve

import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4

# prepare data
dftrain = pd.read_csv('../data/train.csv')
dftest = pd.read_csv('../data/test.csv')

# specify index, predictors and target
index  = 'Id'
target = 'earn_over_4k_euros_per_year'
predictors = [x for x in dftrain.columns if x not in [target,index]]
# base random forest classifier to compare missing value methods
rfc = RandomForestClassifier()

# convert categorical data
continuous_predictors = ['age','income_from_investment_sources',\
                         'losses_from_investment_sources',\
                         'number_of_years_of_education',\
                         'working_hours_per_week']
categorical_predictors = [col for col in predictors if col not in continuous_predictors]

for col in categorical_predictors:
    dftrain[col] = dftrain[col].astype('category')
    dftest[col] = dftrain[col].astype('category')

# fill in missing values
dftrain_ffill = dftrain.fillna(method='ffill')
dftest_ffill = dftest.fillna(method='ffill')
# fill the first row using the last row
dftrain_ffill = dftrain_ffill.fillna(value=dftrain_ffill.iloc[-1])
dftest_ffill = dftrain_ffill.fillna(value=dftest_ffill.iloc[-1]

In [ ]:
# Helper functions
def modelfit(estimator, model, dftrain, predictors, useTrainCV=True, cv_folds=10):
    """fit model with evaluation"""
    # cv
    tic = time.clock()
    cv_scores = cross_validation.cross_val_score(
        estimator=estimator, \
        X=dftrain[predictors].values, \
        y=dftrain[target].values, \
        scoring='accuracy',
        cv=10)
    toc = time.clock()
    
    print 'Model report ({})'.format(model)
    print 'Accuracy (CV): {}'.format(np.mean(cv_scores))
    print 'Time (Avg.): {}'.format((toc - tic) / cv_folds)
    
    # fit
    predictor = estimator.fit(dftrain[predictors].values, dftrain[target].values)
    
    return predictor

In [ ]:
from sklearn.ensemble import RandomForestClassifier, \
GradientBoostingClassifier, AdaBoostClassifier

In [ ]:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier